# Write a program to demonstrate POS tagging: Hidden markov model.

In [2]:
# importing numpy for matrix operations.
import numpy as np

# The provided code defines a class called HMM_POS_Tagging that implements a Hidden Markov Model (HMM) for Part-of-Speech (POS) tagging.
class HMM_POS_Tagging:

    def __init__(self, states, observations, transition_prob, emission_prob, initial_prob): # constructor method for the class.

        self.states = states # A list of possible hidden states in the HMM.
        self.observations = observations # A list of possible observations that can be emitted by the HMM
        self.transition_prob = transition_prob # A 2D array representing the transition probabilities between states.
        self.emission_prob = emission_prob # A 2D array representing the emission probabilities for each state
        self.initial_prob = initial_prob # A 1D array representing the initial state probabilities.

    def viterbi(self, obs_sequence):
        T = len(obs_sequence) # Calculates the length of the observation sequence.
        N = len(self.states) # Calculates the number of possible hidden states.
        viterbi=np.zeros((N,T))

        #  Initializes a 2D NumPy array called viterbi with dimensions equal to the number of states and the length of the observation sequence
        backpointers = np.zeros((N, T), dtype=int)
        # Initializes a 2D NumPy array called backpointers with dimensions equal to the number of states and the length of the observation sequence

        for i in range(N):
            viterbi[i][0] = self.initial_prob[i] * self.emission_prob[i][self.observations.index(obs_sequence[0])]
            # This line calculates the initial Viterbi probability for each state. It multiplies the initial probability of the state (self.initial_prob[i]) with the emission probability of the first observation (obs_sequence[0]) given that state (self.emission_prob[i]

            backpointers[i][0] = 0 # This line initializes the backpointer for the first time step as 0 for all states
            for t in range(1, T): # (index 1). This loop calculates the Viterbi probabilities for the remaining time steps.
              for s in range(N):

                viterbi[s][t] = max(viterbi[prev_s][t - 1] * self.transition_prob[prev_s][s] *
                                    self.emission_prob[s][self.observations.index(obs_sequence[t])]
                                    for prev_s in range(N))

                """ viterbi[s][t]: This is the probability of the most likely sequence of hidden states up to time t, ending in state s.
                max(...): This is a loop that iterates over all possible previous states prev_s at time t-1.

                viterbi[prev_s][t - 1]: This is the probability of the most likely sequence of hidden states up to time t-1, ending in state prev_s.

                self.transition_prob[prev_s][s]: This is the probability of transitioning from state prev_s to state s.

                self.emission_prob[s][self.observations.index(obs_sequence[t])]: This is the probability of observing obs_sequence[t] given that the current state is s.

                obs_sequence[t]: This is the observation at time t.

                """

                backpointers[s][t] = np.argmax([viterbi[prev_s][t - 1] * self.transition_prob[prev_s][s]
                                                 for prev_s in range(N)])
                # viterbi[prev_s][t - 1]: This represents the probability of being in state prev_s at time t - 1.
                # self.transition_prob[prev_s][s]: This represents the transition probability from state prev_s to state s.
                # The np.argmax function is used to find the index of the maximum value in the list [viterbi[prev_s][t - 1] * self.transition_prob[prev_s][s]]

                best_path_prob = max(viterbi[s][T - 1] for s in range(N))

                """ viterbi[s][T - 1]: This represents the probability of being in state s at the final time step, T - 1.
                    for s in range(N): This iterates through all possible states, s, at time T - 1.
                    max: This function returns the maximum value in the list [viterbi[s][T - 1] for s in range(N)].
                """

        best_path_pointer = np.argmax([viterbi[s][T - 1] for s in range(N)])
        best_path = [self.states[best_path_pointer]]

        """np.argmax([viterbi[s][T - 1] for s in range(N)]): This line finds the index of the most likely state at the final time step, T - 1.
            best_path_pointer: This variable stores the index of the most likely final state.
              best_path = [self.states[best_path_pointer]]: This line initializes the list of most likely states with the most likely final state.
        """
        for t in range(T - 1, 0, -1):
            best_path_pointer = backpointers[best_path_pointer][t]
            best_path.insert(0, self.states[best_path_pointer])

            """ for t in range(T - 1, 0, -1): This loop iterates backwards through time, starting from the final time step T - 1 and ending at time step 1.
              best_path_pointer = backpointers[best_path_pointer][t]: This line uses the backpointers matrix to find the most likely previous state at time t given the current most likely state, best_path_pointer.
                best_path.insert(0, self.states[best_path_pointer]): This line adds the most likely state at time t to the beginning of the best_path list.
            """

        return best_path, best_path_prob

# Program

In [2]:
states = ['Noun', 'Verb', 'Adjective']
observations = ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
transition_prob = np.array([[0.3, 0.3, 0.4],
                             [0.1, 0.6, 0.3],
                             [0.2, 0.4, 0.4]])
emission_prob = np.array([[0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                           [0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1],
                           [0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1]])
initial_prob = np.array([0.3, 0.3, 0.4])

hmm = HMM_POS_Tagging(states, observations, transition_prob, emission_prob, initial_prob)
obs_sequence = ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
best_path, best_path_prob = hmm.viterbi(obs_sequence)

print("Best Path (POS Tags):", best_path)
print("Probability of Best Path:", best_path_prob)

Best Path (POS Tags): ['Adjective', 'Verb', 'Verb', 'Verb', 'Verb', 'Verb', 'Verb', 'Verb', 'Verb']
Probability of Best Path: 1.791590400000001e-11
